<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Gumpy-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Testing for EEG | ML 294**

In [5]:
!pip install gumpy numpy scipy matplotlib

In [0]:
from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys
sys.path.append('../../gumpy')

import gumpy
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [7]:
sys.path.append('..')
import models

ModuleNotFoundError: ignored